In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

import anndata as ad
import numpy as np
import pandas as pd

from rich import print


from sklearn.neighbors import NearestNeighbors


import scvi

In [3]:
!ls /home/nathanlevy/Data/

adata_MERFISH_24w.h5ad	adata_scvi_merfish4w.h5ad


In [4]:
data_dir = "/home/nathanlevy/Data/"
file = "adata_scvi_merfish4w.h5ad"

In [5]:
adata = ad.read_h5ad(data_dir + file)
print(adata)

AnnData object with n_obs × n_vars = 123632 × 374
    obs: 'fov', 'center_x', 'center_y', 'min_x', 'max_x', 'min_y', 'max_y', 'age', 'clust_annot', 'slice', 
'organism_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'cell_type_ontology_term_id', 
'assay_ontology_term_id', 'tissue_ontology_term_id', 'disease_ontology_term_id', 
'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'is_primary_data', 
'cell_type_annot', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 
'development_stage', 'donor_slice', 'n_counts', 'cell_index', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'cell_type_colors', 'donor_id_colors', 'log1p', 'neighbors', 'pca', 
'umap'
    obsm: 'X_pca', 'X_scVI', 'X_spVI', 'X_spatial', 'X_umap', 'centroids', 'neighborhood_composition', 'spatial'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

## First train scVI

In [26]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor_id",
)


vae = scvi.model.SCVI(
    adata,
    gene_likelihood="poisson",
    n_layers=1,
    n_latent=10,
)

vae.train(
    max_epochs=2,
    train_size=0.8,
    validation_size=0.2,
    plan_kwargs=dict(n_epochs_kl_warmup=80),
    early_stopping=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 2/2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.31s/it, v_num=1, train_loss_step=181, train_loss_epoch=180]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 2/2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.33s/it, v_num=1, train_loss_step=181, train_loss_epoch=180]


In [27]:
adata.obsm["qz1_m"], adata.obsm["qz1_var"] = vae.get_latent_representation(
    return_dist=True
)

In [8]:
vae.summary_stats

n_batch: 4
n_cells: 123632
n_extra_categorical_covs: 0
n_extra_continuous_covs: 0
n_labels: 1
n_vars: 374

## Then train nichescVI

In [28]:
scvi.model.nicheSCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor_id",
    niche_composition_key="neighborhood_composition",
    niche_indexes_key="niche_indexes",
    sample_key="donor_slice",
    cell_coordinates_key="centroids",
    k_nn=10,
    latent_mean_key="qz1_m",
    latent_var_key="qz1_var",
)

n_cell_types = adata.obsm["neighborhood_composition"].shape[1]

nichevae = scvi.model.nicheSCVI(
    adata,
    k_nn=10,
    n_latent_z1=10,
    niche_kl_weight=1.0,
    gene_likelihood="poisson",
    n_layers=1,
    n_latent=10,
)

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [29]:
print(nichevae.module.niche_decoder)

NicheDecoder(
  (decoder): FCLayers(
    (fc_layers): Sequential(
      (Layer 0): Sequential(
        (0): Linear(in_features=10, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): None
        (3): ReLU()
        (4): None
      )
    )
  )
  (mean_decoder): Linear(in_features=128, out_features=100, bias=True)
  (var_decoder): Linear(in_features=128, out_features=100, bias=True)
)

In [11]:
adata.obsm["niche_indexes"][0]

array([25., 17., 30., 43., 10., 36., 35., 39., 42., 19.])

In [36]:
nichevae.device

device(type='cpu')

In [40]:
latent_cell = torch.ones((2,10), device=nichevae.device)

niche_mean, niche_var = nichevae.module.niche_decoder(latent_cell)

In [41]:
niche_var.shape

torch.Size([2, 100])

In [12]:
nichevae.train(
    max_epochs=1,
    train_size=0.8,
    validation_size=0.2,
    plan_kwargs=dict(n_epochs_kl_warmup=80),
    early_stopping=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1:   0%|                                                                                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

/home/nathanlevy/mambaforge/envs/phenospace_23/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('niche_kl_train', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Epoch 1/1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.91s/it, v_num=1, train_loss_step=179, train_loss_epoch=191]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.91s/it, v_num=1, train_loss_step=179, train_loss_epoch=191]


## Code to train compo-scVI

In [13]:
scvi.model.compoSCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor_id",
    niche_composition_key="neighborhood_composition",
)

n_cell_types = adata.obsm["neighborhood_composition"].shape[1]

compovae = scvi.model.compoSCVI(
    adata,
    gene_likelihood="poisson",
    n_layers=1,
    n_latent=15,
    n_cell_types=n_cell_types,
    ce_weight=10,
)

compovae.train(
    max_epochs=1,
    train_size=0.8,
    validation_size=0.2,
    plan_kwargs=dict(n_epochs_kl_warmup=80),
    early_stopping=False,
)

adata.obsm["ct_pred"] = compovae.predict_neighborhood(softmax=True)

INFO     Generating sequential column names                                                                        


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.21s/it, v_num=1, train_loss_step=183, train_loss_epoch=211]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.21s/it, v_num=1, train_loss_step=183, train_loss_epoch=211]
